In [7]:
import numpy as np
import itertools
from collections import Counter
import multiprocessing as mp
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os

class FranklAdversarialSearch:
    def __init__(self, ground_set_size=6, max_iterations=10000, population_size=100, 
                 tournament_size=5, mutation_rate=0.3, crossover_rate=0.7, 
                 early_stopping=1000, use_multiprocessing=True, verbose=True):
        self.ground_set_size = ground_set_size
        self.max_iterations = max_iterations
        self.population_size = population_size
        self.tournament_size = tournament_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.early_stopping = early_stopping
        self.use_multiprocessing = use_multiprocessing
        self.verbose = verbose
        
        # Results tracking
        self.best_family = None
        self.best_max_frequency = 1.0  # Start with worst case (all elements in all sets)
        self.history = {
            'best_frequencies': [],
            'avg_frequencies': [],
            'family_sizes': [],
            'diversity': []
        }
        
        # Caches to improve performance
        self._union_cache = {}
        self._frequency_cache = {}
        self._metrics_cache = {}
    
    def is_union_closed(self, family):
        """Check if a family of sets is union-closed."""
        # Use a set for O(1) lookups
        family_set = set(family)
        
        for set1, set2 in itertools.combinations(family, 2):
            # Create a cache key for this pair
            cache_key = (set1, set2)
            if cache_key in self._union_cache:
                union = self._union_cache[cache_key]
            else:
                union = tuple(sorted(set(set1).union(set(set2))))
                self._union_cache[cache_key] = union
                
            if union not in family_set:
                return False
        return True
    
    def get_union_closure(self, seed_sets):
        """Generate the union closure of a collection of sets."""
        if not seed_sets:
            return []
            
        # Convert sets to frozensets for hashability
        closure = set(frozenset(s) for s in seed_sets if s)  # Skip empty sets
        
        # Add the empty set if it's in seed_sets
        if () in seed_sets:
            closure.add(frozenset())
            
        old_size = 0
        
        # Keep adding unions until no new sets are formed
        while old_size != len(closure):
            old_size = len(closure)
            new_sets = set()
            
            # Use a list to avoid modifying the set during iteration
            closure_list = list(closure)
            for i in range(len(closure_list)):
                for j in range(i, len(closure_list)):
                    set1 = closure_list[i]
                    set2 = closure_list[j]
                    
                    # Use cache for performance
                    cache_key = (frozenset(set1), frozenset(set2))
                    if cache_key in self._union_cache:
                        union = self._union_cache[cache_key]
                    else:
                        union = set1.union(set2)
                        self._union_cache[cache_key] = union
                        
                    new_sets.add(union)
                    
            closure.update(new_sets)
        
        # Convert back to tuples for easier handling
        return [tuple(sorted(s)) for s in closure]
    
    def calculate_element_frequencies(self, family):
        """Calculate the frequency of each element in the family."""
        # Check cache
        family_key = tuple(sorted(family))
        if family_key in self._frequency_cache:
            return self._frequency_cache[family_key]
            
        element_counts = Counter()
        for s in family:
            element_counts.update(s)
        
        family_size = max(1, len(family))  # Avoid division by zero
        frequencies = {element: count / family_size for element, count in element_counts.items()}
        
        # Update cache
        self._frequency_cache[family_key] = frequencies
        return frequencies
    
    def evaluate_family(self, family):
        """Evaluate a family using maximum element frequency for the Frankl conjecture.
        A counterexample is any family where max frequency < 0.5."""
        if not family:
            return 1.0
        
        frequencies = self.calculate_element_frequencies(family)
        if not frequencies:
            return 1.0
        
        # Use the maximum element frequency as the sole evaluation metric
        max_frequency = max(frequencies.values()) if frequencies else 1.0
        
        # Store metrics for analysis purposes
        metrics = {
            'max_frequency': max_frequency,
            # These are still calculated for analysis but not used in evaluation
            'min_k_avg': self._get_min_k_average(frequencies, k=3),
            'below_threshold_ratio': self._get_below_threshold_ratio(frequencies),
            'frequency_variance': self._get_frequency_variance(frequencies),
            'structure_complexity': self._get_structure_complexity(family, frequencies)
        }
        
        # Store the component metrics for analysis
        family_key = tuple(sorted(family))
        self._metrics_cache = self._metrics_cache if hasattr(self, '_metrics_cache') else {}
        self._metrics_cache[family_key] = metrics
        
        # Add a small penalty for very large families to encourage diversity in the population
        size_penalty = 0.0001 * (len(family) / 100)
        
        # Return the maximum frequency as the score (lower is better)
        return max_frequency + size_penalty
        
    def _get_min_k_average(self, frequencies, k=3):
        """Calculate the average of the k lowest element frequencies."""
        if not frequencies:
            return 1.0
            
        sorted_freqs = sorted(frequencies.values())
        k = min(k, len(sorted_freqs))
        
        if k == 0:
            return 1.0
            
        return sum(sorted_freqs[:k]) / k
        
    def _get_below_threshold_ratio(self, frequencies, threshold=0.5):
        """Calculate the ratio of elements below the threshold frequency."""
        if not frequencies:
            return 0.0
            
        below_count = sum(1 for freq in frequencies.values() if freq < threshold)
        return 1.0 - (below_count / max(1, len(frequencies)))
        
    def _get_frequency_variance(self, frequencies):
        """Calculate the variance in element frequencies."""
        if not frequencies or len(frequencies) < 2:
            return 0.0
            
        freqs = list(frequencies.values())
        mean = sum(freqs) / len(freqs)
        variance = sum((f - mean) ** 2 for f in freqs) / len(freqs)
        
        # Normalize to [0,1] range with a reasonable assumption about max variance
        return min(1.0, variance * 4)
        
    def _get_structure_complexity(self, family, frequencies):
        """Evaluate the structural complexity of the family."""
        if not family or not frequencies:
            return 1.0
            
        # Consider the distribution of set sizes
        set_sizes = [len(s) for s in family]
        size_variety = len(set(set_sizes)) / max(1, len(family))
        
        # Consider the coverage of the ground set
        ground_set_coverage = len(frequencies) / max(1, self.ground_set_size)
        
        # Consider the balance between family size and ground set size
        size_ratio = min(1.0, len(family) / (2 ** min(10, len(frequencies))))
        
        # Combine these factors (lower is better for our search)
        return 0.4 * (1 - size_variety) + 0.3 * (1 - ground_set_coverage) + 0.3 * size_ratio
    
    def evaluate_population(self, population):
        """Evaluate all families in the population in parallel."""
        if self.use_multiprocessing and len(population) > 10:
            with mp.Pool(processes=mp.cpu_count()) as pool:
                results = pool.map(self._evaluate_single, population)
            return results
        else:
            return [self._evaluate_single(seed_sets) for seed_sets in population]
    
    def _evaluate_single(self, seed_sets):
        """Helper function to evaluate a single set of seed sets."""
        family = self.get_union_closure(seed_sets)
        
        # Only consider non-empty, union-closed families
        if family and self.is_union_closed(family):
            max_frequency = self.evaluate_family(family)
            return (seed_sets, family, max_frequency)
        else:
            return (seed_sets, [], 1.0)  # Invalid family
    
    def mutate_seed_sets(self, seed_sets, mutation_rate=None):
        """Apply mutations to seed sets to explore the space."""
        if mutation_rate is None:
            mutation_rate = self.mutation_rate
            
        new_seed_sets = []
        
        # Adaptive mutation: higher rate for stagnant searches
        if len(self.history['best_frequencies']) > 100:
            if len(set(self.history['best_frequencies'][-100:])) < 5:
                mutation_rate *= 1.5  # Increase mutation if stuck
                
        # Operations: add/remove sets, add/remove elements
        for s in seed_sets:
            if np.random.random() < mutation_rate:
                # Mutate this set
                s_list = list(s)
                
                # Choose a mutation type
                mutation_type = np.random.choice(['add', 'remove', 'swap'])
                
                if mutation_type == 'add' or (not s_list and mutation_type == 'swap'):
                    # Add an element
                    new_element = np.random.randint(1, self.ground_set_size + 1)
                    if new_element not in s_list:
                        s_list.append(new_element)
                        s_list.sort()
                elif mutation_type == 'remove' and s_list:
                    # Remove an element
                    idx_to_remove = np.random.randint(0, len(s_list))
                    s_list.pop(idx_to_remove)
                elif mutation_type == 'swap' and s_list:
                    # Swap an element
                    idx_to_swap = np.random.randint(0, len(s_list))
                    current = s_list[idx_to_swap]
                    new_element = current
                    while new_element == current:
                        new_element = np.random.randint(1, self.ground_set_size + 1)
                    s_list[idx_to_swap] = new_element
                    s_list.sort()
                
                new_seed_sets.append(tuple(s_list))
            else:
                new_seed_sets.append(s)
        
        # Perform set-level mutations
        set_mutation_type = np.random.choice(['add', 'remove', 'merge', 'none'], 
                                           p=[0.3, 0.3, 0.2, 0.2])
        
        if set_mutation_type == 'add':
            # Add a set
            set_size = np.random.randint(1, min(5, self.ground_set_size) + 1)
            new_set = tuple(sorted(np.random.choice(
                range(1, self.ground_set_size + 1), 
                size=min(set_size, self.ground_set_size),
                replace=False)))
            if new_set and new_set not in new_seed_sets:
                new_seed_sets.append(new_set)
        elif set_mutation_type == 'remove' and len(new_seed_sets) > 1:
            # Remove a set
            idx_to_remove = np.random.randint(0, len(new_seed_sets))
            new_seed_sets.pop(idx_to_remove)
        elif set_mutation_type == 'merge' and len(new_seed_sets) >= 2:
            # Merge two sets
            idx1, idx2 = np.random.choice(len(new_seed_sets), size=2, replace=False)
            set1 = set(new_seed_sets[idx1])
            set2 = set(new_seed_sets[idx2])
            merged = tuple(sorted(set1.union(set2)))
            
            # Replace both sets with the merged one
            if idx1 > idx2:
                new_seed_sets.pop(idx1)
                new_seed_sets.pop(idx2)
            else:
                new_seed_sets.pop(idx2)
                new_seed_sets.pop(idx1)
                
            if merged:
                new_seed_sets.append(merged)
        
        # Ensure we have at least one set
        if not new_seed_sets:
            new_element = np.random.randint(1, self.ground_set_size + 1)
            new_seed_sets.append((new_element,))
            
        return new_seed_sets
    
    def crossover(self, parent1, parent2):
        """Perform crossover between two families' seed sets."""
        if not parent1 or not parent2:
            return parent1 if parent1 else parent2
            
        # Choose crossover type
        crossover_type = np.random.choice(['one_point', 'two_point', 'uniform'])
        
        if crossover_type == 'one_point':
            # One-point crossover
            cut_point = np.random.randint(0, min(len(parent1), len(parent2)) + 1)
            child = parent1[:cut_point] + parent2[cut_point:]
        elif crossover_type == 'two_point' and min(len(parent1), len(parent2)) > 2:
            # Two-point crossover
            points = sorted(np.random.choice(range(min(len(parent1), len(parent2)) + 1), 
                                          size=2, replace=False))
            child = parent1[:points[0]] + parent2[points[0]:points[1]] + parent1[points[1]:]
        else:
            # Uniform crossover
            child = []
            for i in range(max(len(parent1), len(parent2))):
                if i < len(parent1) and i < len(parent2):
                    # Choose from either parent
                    child.append(parent1[i] if np.random.random() < 0.5 else parent2[i])
                elif i < len(parent1):
                    # Only parent1 has this index
                    if np.random.random() < 0.5:  # 50% chance to include
                        child.append(parent1[i])
                else:
                    # Only parent2 has this index
                    if np.random.random() < 0.5:  # 50% chance to include
                        child.append(parent2[i])
        
        # Remove duplicates while preserving order
        seen = set()
        unique_child = []
        for item in child:
            if item not in seen:
                seen.add(item)
                unique_child.append(item)
                
        # Ensure we have at least one set
        if not unique_child:
            if parent1:
                unique_child.append(parent1[0])
            elif parent2:
                unique_child.append(parent2[0])
            else:
                new_element = np.random.randint(1, self.ground_set_size + 1)
                unique_child.append((new_element,))
                
        return unique_child
    
    def calculate_population_diversity(self, population):
        """Calculate diversity metric for the population."""
        if not population:
            return 0
            
        # Use a simple measure: the average pairwise difference between individuals
        total_diff = 0
        count = 0
        
        # Sample pairs to avoid O(n²) computation for large populations
        sample_size = min(100, len(population) * (len(population) - 1) // 2)
        pairs = np.random.choice(len(population), size=(sample_size, 2), replace=True)
        
        for i, j in pairs:
            if i != j:
                # Count the number of different sets
                set1 = set(population[i])
                set2 = set(population[j])
                diff = len(set1.symmetric_difference(set2)) / max(1, len(set1) + len(set2))
                total_diff += diff
                count += 1
                
        return total_diff / max(1, count)
    
    def search(self):
        """Use a genetic algorithm approach to search for families with max element frequency < 0.5."""
        start_time = time.time()
        no_improvement_count = 0
        
        # Initialize population with random seed sets
        population = []
        for _ in range(self.population_size):
            num_sets = np.random.randint(2, min(8, self.ground_set_size // 2))
            seed_sets = []
            for _ in range(num_sets):
                set_size = np.random.randint(1, min(5, self.ground_set_size) + 1)
                new_set = tuple(sorted(np.random.choice(
                    range(1, self.ground_set_size + 1), 
                    size=min(set_size, self.ground_set_size),
                    replace=False)))
                if new_set:  # Ensure non-empty
                    seed_sets.append(new_set)
            if not seed_sets:  # Ensure at least one set
                new_element = np.random.randint(1, self.ground_set_size + 1)
                seed_sets.append((new_element,))
            population.append(seed_sets)
        
        # Create a progress bar if verbose
        iterator = tqdm(range(self.max_iterations)) if self.verbose else range(self.max_iterations)
        
        for iteration in iterator:
            # Evaluate population
            evaluation_results = self.evaluate_population(population)
            
            # Extract frequencies and update best solution
            frequencies = [freq for _, _, freq in evaluation_results]
            seed_sets_list = [seeds for seeds, _, _ in evaluation_results]
            family_list = [family for _, family, _ in evaluation_results]
            
            # Track metrics
            avg_frequency = sum(frequencies) / len(frequencies)
            self.history['avg_frequencies'].append(avg_frequency)
            
            # Find the best solution in this generation
            min_idx = np.argmin(frequencies)
            current_best_freq = frequencies[min_idx]
            current_best_family = family_list[min_idx]
            
            if current_best_freq < self.best_max_frequency and current_best_family:
                self.best_max_frequency = current_best_freq
                self.best_family = current_best_family
                no_improvement_count = 0
                
                if self.verbose:
                    print(f"\nIteration {iteration}, New best: {current_best_freq:.4f}")
                    print(f"Family size: {len(self.best_family)}")
                    element_freqs = self.calculate_element_frequencies(self.best_family)
                    max_element = max(element_freqs.items(), key=lambda x: x[1])
                    print(f"Max frequency element: {max_element[0]} with frequency {max_element[1]:.4f}")
                    print(f"Distance from counterexample threshold: {(max_element[1] - 0.5) * 100:.2f}%")
                    
                # If we find a counterexample, stop immediately and report it
                if current_best_freq < 0.5:
                    print("\n========================================")
                    print("COUNTEREXAMPLE TO FRANKL'S CONJECTURE FOUND!")
                    print("========================================")
                    print(f"Maximum element frequency: {current_best_freq:.6f}")
                    print(f"Family size: {len(self.best_family)}")
                    print(f"Ground set size: {self.ground_set_size}")
                    
                    # Display the actual counterexample
                    print("\nCounterexample family:")
                    for i, s in enumerate(sorted(self.best_family, key=len)):
                        print(f"Set {i+1}: {s}")
                        
                    # Print the element frequencies
                    element_freqs = self.calculate_element_frequencies(self.best_family)
                    print("\nElement frequencies:")
                    for element, freq in sorted(element_freqs.items(), key=lambda x: x[1], reverse=True):
                        print(f"Element {element}: {freq:.6f}")
                        
                    return self.best_family
            else:
                no_improvement_count += 1
            
            # Record history
            self.history['best_frequencies'].append(self.best_max_frequency)
            
            if self.best_family:
                self.history['family_sizes'].append(len(self.best_family))
            else:
                self.history['family_sizes'].append(0)
                
            diversity = self.calculate_population_diversity(seed_sets_list)
            self.history['diversity'].append(diversity)
            
            # Check for early stopping
            if no_improvement_count >= self.early_stopping:
                if self.verbose:
                    print(f"\nEarly stopping after {no_improvement_count} iterations without improvement")
                break
                
            # Create new population
            new_population = []
            
            # Elitism: keep the best solutions
            elite_count = max(1, self.population_size // 20)
            elite_indices = np.argsort(frequencies)[:elite_count]
            for idx in elite_indices:
                new_population.append(seed_sets_list[idx])
            
            # Fill the rest with tournament selection and crossover
            while len(new_population) < self.population_size:
                # Tournament selection
                tournament = np.random.choice(len(population), self.tournament_size, replace=False)
                tournament_freqs = [frequencies[i] for i in tournament]
                
                # Select the best two parents
                parent_indices = np.argsort(tournament_freqs)[:2]
                parent1_idx = tournament[parent_indices[0]]
                parent2_idx = tournament[parent_indices[1]]
                
                parent1 = seed_sets_list[parent1_idx]
                parent2 = seed_sets_list[parent2_idx]
                
                # Crossover with probability
                if np.random.random() < self.crossover_rate:
                    child = self.crossover(parent1, parent2)
                else:
                    child = parent1.copy() if np.random.random() < 0.5 else parent2.copy()
                
                # Mutation
                child = self.mutate_seed_sets(child)
                
                new_population.append(child)
            
            population = new_population
            
            # Adaptive parameter adjustment
            if iteration > 100 and iteration % 50 == 0:
                # If diversity is low, increase mutation rate
                if self.history['diversity'][-1] < 0.1:
                    self.mutation_rate = min(0.8, self.mutation_rate * 1.2)
                # If progress is stagnant but diversity is high, increase selection pressure
                elif len(set(self.history['best_frequencies'][-50:])) < 3:
                    self.tournament_size = min(10, self.tournament_size + 1)
                    
            # Update progress bar with status
            if self.verbose and isinstance(iterator, tqdm):
                iterator.set_description(f"Best: {self.best_max_frequency:.4f}, Avg: {avg_frequency:.4f}")
        
        # Display final statistics
        elapsed_time = time.time() - start_time
        if self.verbose:
            print(f"\nSearch completed in {elapsed_time:.2f} seconds")
            print(f"Total iterations: {min(iteration + 1, self.max_iterations)}")
            print(f"Best frequency found: {self.best_max_frequency:.4f}")
            
        # Return the best family found
        return self.best_family
    
    def plot_results(self):
        """Plot comprehensive search history and analysis."""
        if not self.history['best_frequencies']:
            print("No search history to plot.")
            return
            
        plt.figure(figsize=(16, 12))
        
        # 1. Plot metrics evolution
        plt.subplot(3, 2, 1)
        plt.plot(self.history['best_frequencies'], label='Best Score', color='blue')
        plt.plot(self.history['avg_frequencies'], label='Avg Score', color='lightblue', alpha=0.7)
        plt.axhline(y=0.5, color='r', linestyle='--', label='Conjecture Threshold')
        plt.xlabel('Iteration')
        plt.ylabel('Score')
        plt.legend()
        plt.title('Search Progress')
        
        # 2. Plot family size evolution
        plt.subplot(3, 2, 2)
        plt.plot(self.history['family_sizes'], color='green')
        plt.xlabel('Iteration')
        plt.ylabel('Family Size')
        plt.title('Best Family Size Evolution')
        
        # 3. Plot population diversity
        plt.subplot(3, 2, 3)
        plt.plot(self.history['diversity'], color='purple')
        plt.xlabel('Iteration')
        plt.ylabel('Diversity')
        plt.title('Population Diversity')
        
        # 4. Element frequency distribution for the best family
        if self.best_family:
            plt.subplot(3, 2, 4)
            frequencies = self.calculate_element_frequencies(self.best_family)
            
            # Create histogram of frequencies
            freq_values = list(frequencies.values())
            plt.hist(freq_values, bins=20, color='teal', alpha=0.7)
            plt.axvline(x=0.5, color='r', linestyle='--', label='Threshold')
            plt.xlabel('Frequency')
            plt.ylabel('Count')
            plt.title('Distribution of Element Frequencies')
            plt.legend()
            
            # 5. Set size distribution
            plt.subplot(3, 2, 5)
            set_sizes = [len(s) for s in self.best_family]
            size_counts = Counter(set_sizes)
            sizes = sorted(size_counts.keys())
            counts = [size_counts[s] for s in sizes]
            
            plt.bar(sizes, counts, color='orange')
            plt.xlabel('Set Size')
            plt.ylabel('Count')
            plt.title('Set Size Distribution')
            
            # 6. Sorted element frequencies
            plt.subplot(3, 2, 6)
            sorted_elements = sorted(frequencies.items(), key=lambda x: x[1])
            elements = [e for e, _ in sorted_elements]
            freqs = [f for _, f in sorted_elements]
            
            plt.scatter(range(len(elements)), freqs, color='blue', alpha=0.7)
            plt.axhline(y=0.5, color='r', linestyle='--')
            plt.xlabel('Element Index (sorted by frequency)')
            plt.ylabel('Frequency')
            plt.title('Sorted Element Frequencies')
            
            # Add text annotation for the maximum frequency
            max_freq = max(freqs)
            plt.annotate(f'Max: {max_freq:.4f}', 
                         xy=(len(freqs)-1, max_freq),
                         xytext=(len(freqs)-len(freqs)//4, max_freq+0.1),
                         arrowprops=dict(facecolor='black', shrink=0.05, width=1.5))
        
        plt.tight_layout()
        
        # Create a second figure for additional analysis
        if self.best_family and len(self.best_family) > 0:
            plt.figure(figsize=(16, 8))
            
            # 1. Heatmap of set membership
            plt.subplot(1, 2, 1)
            frequencies = self.calculate_element_frequencies(self.best_family)
            elements = sorted(frequencies.keys())
            
            # Create matrix: rows=sets, cols=elements
            matrix = np.zeros((min(50, len(self.best_family)), len(elements)))
            for i, s in enumerate(self.best_family[:50]):  # Limit to 50 sets for visibility
                for j, e in enumerate(elements):
                    matrix[i, j] = 1 if e in s else 0
            
            # Plot heatmap
            sns.heatmap(matrix, cmap='Blues', cbar=False)
            plt.xlabel('Elements')
            plt.ylabel('Sets (first 50)')
            plt.title('Set Membership Matrix')
            
            # 2. Network visualization of element co-occurrence
            plt.subplot(1, 2, 2)
            
            # Count co-occurrences
            co_occurrences = np.zeros((len(elements), len(elements)))
            for s in self.best_family:
                for i, e1 in enumerate(elements):
                    for j, e2 in enumerate(elements):
                        if e1 in s and e2 in s:
                            co_occurrences[i, j] += 1
            
            # Normalize and visualize
            co_occurrences = co_occurrences / len(self.best_family)
            mask = np.triu(np.ones_like(co_occurrences, dtype=bool))
            sns.heatmap(co_occurrences, mask=mask, cmap="YlGnBu", vmin=0, vmax=1)
            plt.title('Element Co-occurrence')
            
            plt.tight_layout()
        
        plt.show()
    
    def analyze_best_family(self):
        """Analyze the best family found with comprehensive metrics."""
        if not self.best_family:
            print("No family found.")
            return
            
        print("\nBest Family Analysis:")
        print(f"Family size: {len(self.best_family)}")
        
        # Analyze element frequencies
        frequencies = self.calculate_element_frequencies(self.best_family)
        max_freq = max(frequencies.values()) if frequencies else 0
        print(f"Maximum element frequency: {max_freq:.4f}")
        
        # Calculate all metrics
        family_key = tuple(sorted(self.best_family))
        if hasattr(self, '_metrics_cache') and family_key in self._metrics_cache:
            metrics = self._metrics_cache[family_key]
        else:
            metrics = {
                'min_k_avg': self._get_min_k_average(frequencies, k=3),
                'below_threshold_ratio': self._get_below_threshold_ratio(frequencies),
                'frequency_variance': self._get_frequency_variance(frequencies),
                'structure_complexity': self._get_structure_complexity(self.best_family, frequencies),
                'max_frequency': max_freq
            }
        
        print("\nMetrics Summary:")
        print(f"Average of 3 lowest frequencies: {metrics['min_k_avg']:.4f}")
        below_count = sum(1 for freq in frequencies.values() if freq < 0.5)
        print(f"Elements below 0.5 threshold: {below_count}/{len(frequencies)} ({(1-metrics['below_threshold_ratio'])*100:.1f}%)")
        print(f"Frequency variance: {metrics['frequency_variance']:.4f}")
        print(f"Structure complexity score: {metrics['structure_complexity']:.4f}")
        
        # Frequency distribution visualization
        print("\nElement frequencies:")
        sorted_freqs = sorted(frequencies.items(), key=lambda x: x[1])
        
        # Print a simple histogram of frequencies
        freq_ranges = {'0.0-0.1': 0, '0.1-0.2': 0, '0.2-0.3': 0, '0.3-0.4': 0, 
                      '0.4-0.5': 0, '0.5-0.6': 0, '0.6-0.7': 0, '0.7-0.8': 0, 
                      '0.8-0.9': 0, '0.9-1.0': 0}
        
        for _, freq in frequencies.items():
            range_idx = min(int(freq * 10), 9)
            range_key = f"{range_idx/10:.1f}-{(range_idx+1)/10:.1f}"
            freq_ranges[range_key] += 1
            
        print("Frequency distribution:")
        for range_key, count in freq_ranges.items():
            bar = "#" * (count * 50 // max(1, len(frequencies)))
            print(f"{range_key}: {count} {bar}")
        
        # Detailed top and bottom elements
        print("\nTop 5 highest frequency elements:")
        for element, freq in sorted(frequencies.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"Element {element}: {freq:.4f}")
            
        print("\nTop 5 lowest frequency elements:")
        for element, freq in sorted_freqs[:5]:
            print(f"Element {element}: {freq:.4f}")
            
        # Analyze set sizes
        set_sizes = [len(s) for s in self.best_family]
        size_counts = Counter(set_sizes)
        print("\nSet size distribution:")
        for size, count in sorted(size_counts.items()):
            bar = "*" * (count * 50 // len(self.best_family))
            print(f"Size {size}: {count} sets ({count/len(self.best_family)*100:.1f}%) {bar}")
            
        # Find minimal generating sets
        min_generating_size = float('inf')
        
        # Start with small random samples
        for sample_size in range(1, min(len(self.best_family), 10)):
            for _ in range(100):  # Try multiple random samples
                sample = list(np.random.choice(len(self.best_family), sample_size, replace=False))
                sample_sets = [self.best_family[i] for i in sample]
                closure = self.get_union_closure(sample_sets)
                
                # Check if this sample generates the whole family
                if set(tuple(sorted(s)) for s in closure) == set(tuple(sorted(s)) for s in self.best_family):
                    min_generating_size = min(min_generating_size, sample_size)
                    break
            
            if min_generating_size <= sample_size:
                break
                
        print(f"\nMinimum generating set size (estimate): {min_generating_size}")
        
        # Analyze elements that appear in most of the smaller sets
        small_sets = [s for s in self.best_family if len(s) <= 3]
        if small_sets:
            small_set_elements = Counter()
            for s in small_sets:
                small_set_elements.update(s)
            
            print("\nMost common elements in small sets (size ≤ 3):")
            for element, count in small_set_elements.most_common(5):
                print(f"Element {element}: appears in {count}/{len(small_sets)} small sets ({count/len(small_sets)*100:.1f}%)")
        
        # Check if this is close to a counterexample
        threshold = 0.5
        elements_below = [element for element, freq in frequencies.items() if freq < threshold]
        if elements_below:
            print(f"\nElements below {threshold} threshold: {len(elements_below)}/{len(frequencies)}")
            if max_freq < 0.55:
                print("\nThis family is close to a potential counterexample!")
                print(f"The gap to the conjecture threshold is only {(max_freq - 0.5) * 100:.2f}%")
                
                # Analyze which sets contain the elements with lowest frequencies
                low_elements = [element for element, freq in sorted_freqs[:3]]
                print("\nAnalyzing sets containing the lowest frequency elements:")
                for element in low_elements:
                    containing_sets = [s for s in self.best_family if element in s]
                    print(f"Element {element} (freq: {frequencies[element]:.4f}) appears in {len(containing_sets)}/{len(self.best_family)} sets")
                    
                    # Show some example sets
                    if containing_sets:
                        print("Example sets:")
                        for i, s in enumerate(containing_sets[:3]):
                            print(f"  Set {i+1}: {s}")
                        if len(containing_sets) > 3:
                            print(f"  ... and {len(containing_sets)-3} more sets")

In [8]:
def deep_search(ground_size=16, iterations=15000, population_size=500, use_multiprocessing=True, verbose=True):
    """Run a deeper search with more resources for a specific ground set size."""
    print(f"\n=== Deep search with ground set size {ground_size} ===")
    print(f"Parameters: iterations={iterations}, population_size={population_size}")
    
    searcher = FranklAdversarialSearch(
        ground_set_size=ground_size,
        max_iterations=iterations,
        population_size=population_size,
        tournament_size=10,                # Increased tournament size for higher selection pressure
        mutation_rate=0.3,
        crossover_rate=0.8,
        early_stopping=2000,               # Increased early stopping threshold for thorough search
        use_multiprocessing=use_multiprocessing,
        verbose=verbose
    )
    
    result = searcher.search()
    
    print(f"\nDeep search completed.")
    if searcher.best_max_frequency < 0.5:
        print("COUNTEREXAMPLE TO FRANKL'S CONJECTURE FOUND!")
    else:
        print("No counterexample found. This supports the conjecture.")
        print(f"Best maximum element frequency found: {searcher.best_max_frequency:.6f}")
        print(f"Gap to counterexample threshold: {(searcher.best_max_frequency - 0.5) * 100:.4f}%")
    
    # Perform detailed analysis
    print("\nPerforming detailed analysis of the best family found...")
    searcher.analyze_best_family()
    searcher.plot_results()
    
    return searcher

In [9]:
def random_search(ground_set_size=10, num_attempts=1000, max_family_size=50, verbose=True):
    """
    Perform a random search for counterexamples to Frankl's conjecture.
    
    This function generates random union-closed families and checks if they
    violate the conjecture (have maximum element frequency < 0.5).
    
    Args:
        ground_set_size: Size of the ground set
        num_attempts: Number of random families to generate
        max_family_size: Maximum number of seed sets to generate for each family
        verbose: Whether to display progress information
    
    Returns:
        The best family found (closest to being a counterexample)
    """
    print(f"\n=== Random search with ground set size {ground_set_size} ===")
    print(f"Parameters: attempts={num_attempts}, max_family_size={max_family_size}")
    
    best_family = None
    best_max_frequency = 1.0
    
    # For calculating element frequencies
    def calculate_frequencies(family):
        if not family:
            return {}
        
        element_counts = Counter()
        for s in family:
            element_counts.update(s)
        
        family_size = len(family)
        return {element: count / family_size for element, count in element_counts.items()}
    
    # For generating random sets
    def generate_random_set():
        set_size = np.random.randint(1, ground_set_size + 1)
        return tuple(sorted(np.random.choice(range(1, ground_set_size + 1), 
                                          size=min(set_size, ground_set_size),
                                          replace=False)))
    
    # Progress tracking
    if verbose:
        iterator = tqdm(range(num_attempts))
    else:
        iterator = range(num_attempts)
    
    for attempt in iterator:
        # Generate a random number of seed sets
        num_sets = np.random.randint(1, max_family_size + 1)
        seed_sets = []
        
        # Generate random sets
        for _ in range(num_sets):
            new_set = generate_random_set()
            if new_set:  # Non-empty
                seed_sets.append(new_set)
        
        # Skip if no sets were generated
        if not seed_sets:
            continue
            
        # Generate the union closure
        closure = set()
        for s in seed_sets:
            closure.add(frozenset(s))
            
        old_size = 0
        while old_size != len(closure):
            old_size = len(closure)
            new_sets = set()
            for set1, set2 in itertools.combinations(closure, 2):
                new_sets.add(frozenset(set1.union(set2)))
            closure.update(new_sets)
        
        # Convert back to tuples for easier handling
        family = [tuple(sorted(s)) for s in closure]
        
        # Calculate maximum element frequency
        frequencies = calculate_frequencies(family)
        max_freq = max(frequencies.values()) if frequencies else 1.0
        
        # Update if this is better than our previous best
        if max_freq < best_max_frequency:
            best_max_frequency = max_freq
            best_family = family
            
            if verbose:
                print(f"\nAttempt {attempt}, New best: {max_freq:.4f}")
                print(f"Family size: {len(family)}")
                max_element = max(frequencies.items(), key=lambda x: x[1]) if frequencies else None
                if max_element:
                    print(f"Max frequency element: {max_element[0]} with frequency {max_element[1]:.4f}")
                print(f"Distance from counterexample threshold: {(max_freq - 0.5) * 100:.2f}%")
                
            # If a counterexample is found, stop immediately
            if max_freq < 0.5:
                print("\n========================================")
                print("COUNTEREXAMPLE TO FRANKL'S CONJECTURE FOUND!")
                print("========================================")
                print(f"Maximum element frequency: {max_freq:.6f}")
                print(f"Family size: {len(family)}")
                print(f"Ground set size: {ground_set_size}")
                break
                
        # Update progress bar
        if verbose and isinstance(iterator, tqdm):
            iterator.set_description(f"Best: {best_max_frequency:.4f}")
    
    # Final report
    print("\nRandom search completed.")
    print(f"Best maximum element frequency found: {best_max_frequency:.6f}")
    print(f"Family size: {len(best_family) if best_family else 0}")
    print(f"Gap to counterexample threshold: {(best_max_frequency - 0.5) * 100:.4f}%")
    
    # Analyze the best family
    if best_family:
        # Create a simple analysis of the best family
        frequencies = calculate_frequencies(best_family)
        print("\nElement frequencies:")
        for element, freq in sorted(frequencies.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"Element {element}: {freq:.4f}")
        
        # Set size distribution
        set_sizes = [len(s) for s in best_family]
        size_counts = Counter(set_sizes)
        print("\nSet size distribution:")
        for size, count in sorted(size_counts.items()):
            print(f"Size {size}: {count} sets ({count/len(best_family)*100:.1f}%)")
    
    return best_family

In [ ]:
# Example usage with random search added:
if __name__ == "__main__":
    # Choose a search method:
    
    # Option 1: Single search with specific parameters
    # searcher = run_frankl_search(ground_set_size=12, iterations=3000, population_size=300)
    
    # Option 2: Systematic search with increasing ground set sizes
    # best_results = systematic_search(ground_sizes=[8, 12, 16], verbose=True)
    
    # Option 3: Deep search on a specific ground set size
     searcher = deep_search(ground_size=70, iterations=10000, population_size=500)
    
    # Option 4: Random search approach
    #best_family = random_search(ground_set_size=70, num_attempts=2000, max_family_size=20, verbose=True)
    
    # If you want to verify a specific family
    # family = [(1, 2), (2, 3), (1, 3), (1, 2, 3)]  # Example family
    # verify_family(family, ground_set_size=3)


=== Deep search with ground set size 70 ===
Parameters: iterations=10000, population_size=500


Best: 0.5041, Avg: 0.6985:   0%|          | 1/10000 [00:09<25:43:51,  9.26s/it]


Iteration 0, New best: 0.5041
Family size: 127
Max frequency element: 8 with frequency 0.5039
Distance from counterexample threshold: 0.39%


Best: 0.5000, Avg: 0.6450:   0%|          | 2/10000 [00:14<18:24:42,  6.63s/it]


Iteration 1, New best: 0.5000
Family size: 2
Max frequency element: 11 with frequency 0.5000
Distance from counterexample threshold: 0.00%


Best: 0.5000, Avg: 0.7149:   9%|▉         | 931/10000 [04:48<46:03,  3.28it/s]  